In [ ]:
import math
import random

<b>Variable aleatoria exponencial</b>
Se utiliza para calcular el tiempo de cada solicitud dentro del servidor.

In [ ]:
def exponencial(lde,t):
    return -((1/lde)*math.log(random.random()))

<b>Variable de proceso de Poisson</b>
Se utiliza para generar los tiempos de ocurrencia, con ella nos movemos en el tiempo dentro de la simulación.

In [ ]:
def poisson(ldpp,t):
    return t-((1/ldpp)*math.log(random.random()))

Se optó por utilizar programación orientada a objetos. A continuación se describen las clases utilizadas.

<b>Task</b><br>
<i>    Atributos:</i>
<ul>
  <li>       t_llegada (float): Tiempo en el que la solicitud llega al sistema (ta).</li>
  <li>       t_atendido (float): Tiempo en que la solicitud es atendida por el servidor.</li>
  <li>       t_salida (float): Tiempo en que la solicitud sale del sistema.</li>
</ul>
<i>    Métodos:</i>
<ul>
  <li>       get_tiempoT: Retorna el tiempo total de la solicitud dentro del sistema.</li>
  <li>       get_tiempoEnServ: Retorna el tiempo que le tomó al servidor atender la solicitud.</li>
</ul>

In [ ]:
class task:
    def __init__(self, ta):
        self.t_llegada = ta
        self.t_atendido = float('Inf')
        self.t_salida = float('Inf')
    
    #devuelve el tiempo total del task
    def get_tiempoT(self):
        return (self.t_salida - self.t_llegada)
    
    #devuelve el tiempo en servidor
    def get_tiempoEnServ(self):
        return (self.t_salida - self.t_atendido)

<b>Server</b><br>
<i>    Atributos:</i>
<ul>
  <li>       bussy (bool): Indica el estado del servidor (ocupado/desocupado)</li>
  <li>       task_running (task): Solicitud que está siendo atendida.</li>
  <li>       tasks (int array): Lista de las solicitudes procesadas por el servidor.</li>
  <li>       Tp (float): Tiempo extra en que estuvo ocupado el servidor.</li>
</ul>
<i>    Métodos:</i>
<ul>
  <li>       assign_task: Se le asigna una tarea a procesar al servidor.</li>
  <li>       drop_task: Se agrega la tarea atendida a la lista tasks y se libera el servidor.</li>
</ul>

In [ ]:
class server:
    def __init__(self):
        self.bussy = False
        self.task_running = False
        self.tasks = [] #processed tasks
        self.Tp = 0
    
    def assign_task(self, taske):
        self.bussy = True
        self.task_running = taske
        
    def drop_task(self):
        self.bussy = False
        self.tasks.append(self.task_running)
        self.task_running = False
        

<b>SystemSim</b><br>
<i>    Atributos:</i>
<ul>
  <li>       serv_working (bool): Indica el estado del sistema (en funcionamiento o no).</li>
  <li>       T (float): Tiempo de simulación.</li>
  <li>       Tp_general (float): Tiempo extra en que funcionó el sistema para atender a todas las solicitudes ingresadas.</li>
  <li>       ldp (float): Lambda del proceso de Poisson.</li>
  <li>       lde (float): Lambda de la variable exponencial.</li>
  <li>       t (float): Tiempo actual.</li>
  <li>       Na (int): Solicitudes ingresadas al tiempo t.</li>
  <li>       Nd (int): Solicitudes procesadas al tiempo t.</li>  
  <li>       ta (float): Tiempo de llegada de la próxima solicitud.</li>
  <li>       cola_eventos (task array): Almacena las solicitudes en cola.</li>  
  <li>       servidores (server array): Lista de servidores del sistema.</li>  
</ul>
<i>    Métodos:</i>
<ul>
  <li>       get_tiempoDisp: Indica si aún queda tiempo de simulación.</li>
  <li>       add_task: Se agrega una nueva solicitud a la cola y si existe algún servidor desocupado se asigna a este para ser procesada.</li>
  <li>       assign_task: Asigna la solicitud a algún servidor desocupado, la elimina de la cola de eventos y le asigna un tiempo de salida.</li>
  <li>       get_free: Indica qué servidor se encuentra libre.</li>
  <li>       get_td: Retorna el tiempo de salida más pequeño.</li>
  <li>       debug: Imprime variables necesarias para debugeo.</li>
</ul>

In [ ]:
class SystemSim:
    def __init__(self, T, ldp, lde, servidores):
        #flags
        self.serv_working = True #cambia hasta el caso 4
        #parametros cambian
        self.T = T
        self.Tp_general = 0
        self.ldp = ldp
        self.lde = lde
        
        #parametros constantes
        self.t = 0
        self.Na = 0
        self.Nd = 0
        self.ta = poisson(ldp,self.t)
        
        #listas
        self.cola_eventos = []
        self.servidores = []
        
        #inicializo objetos servidor
        for i in range(servidores):
            self.servidores.append(server())
            
    #Verifica si aun hay tiempo de ejecucion
    def get_tiempoDisp(self):
        if(self.t < self.T and self.serv_working):
            return True
        else:
            return False
    
    #agrega un nuevo objeto a la cola
    def add_task(self,ta):
        servidor_pos = self.get_free()
        self.cola_eventos.append(task(ta)) #### se mete task a la cola
        #existe un servidor desocupado
        if(servidor_pos > -1):
            self.assign_task(ta,servidor_pos)
    
    def assign_task(self,tx,index):
        self.servidores[index].assign_task(self.cola_eventos[0]) #se agrega al servidor seleccionado
        self.servidores[index].task_running.t_atendido = tx #se asigna el tiempo de atendido
        self.servidores[index].task_running.t_salida = self.t + exponencial(self.lde,self.t) #se asigna tiempo de salida
        self.cola_eventos.pop(0) #task atendiendo
        
    #identifica que servidor esta libre
    def get_free(self):
        #devuelve -1 si no hay libres
        for i in range(len(self.servidores)):
            if(not self.servidores[i].bussy):
                return i
        return -1
    
    #devuelve el tiempo de salida mas pequeño
    def get_td(self):
        td_min = float('Inf')
        td_servidor = -1
        for i in range(len(self.servidores)):
            if((self.servidores[i].task_running != False) and (self.servidores[i].task_running.t_salida < td_min)):
                td_min = self.servidores[i].task_running.t_salida
                td_servidor = i
        return (td_min,td_servidor)
        
    #debug block
    def debug(self):
        print("t: ", self.t, "\tta: ", self.ta, "\ttd: ", self.get_td()[0])
        print("Servidores: ",end='')
        for i in self.servidores:
            print("\t",end='')
            print(i.bussy,end='')
        print("\ttasks_cola: ", end='')
        print(len(self.cola_eventos),end='')
        print("")
        

Los casos para el experimento de un solo servidor que encola fueron ligeramente modificados para la implementación de múltiples servidores.

<b>Caso 1: </b>
El siguiente evento es una llegada de una solicitud al sistema y aún no es la hora de cierre. 

<b>Caso 2: </b>
El siguiente evento es una salida de una solicitud del sistema. Si ocurrre luego de la hora de cierre se suma tiempo extra al servidor. Esta modificación hace prescindible el caso 3. 

<b>Caso 4: </b>
El próximo evento ocurre luego de la hora de cierre y ya no hay solicitudes en cola. 

In [ ]:
def caso1(world):
    #print("CASO 1",end='')
    if(world.ta <= world.T):
        #world.debug()
        world.t = world.ta
        world.Na += 1
        world.ta = poisson(world.ldp,world.t)
        #print("\tNew_ta: ",world.ta)
        world.add_task(world.t)

In [ ]:
def caso2(world):
    #print("CASO 2")
    #world.debug()
    world.t = world.get_td()[0]
    servidor = world.get_td()[1] #es un indice
    world.Nd += 1
    #world.servidores[servidor].bussy = False
    world.servidores[servidor].drop_task()
    
    if(not world.get_tiempoDisp()):
        world.servidores[servidor].Tp += max(world.t - world.T, 0)
        
    if(len(world.cola_eventos)>0):
        world.assign_task(world.t,servidor)
        
        

In [ ]:
def caso4(world):
    #print("CASO 4")
    world.serv_working = False

In [ ]:
def selector(world):
    if((world.ta < world.get_td()[0]) and (world.ta < world.T)):
        # El siguiente evento es una llegada de una solicitud al 
        # sistema y aún no es la hora de cierre. 
        caso1(world)
    elif(world.get_td()[0] < world.ta):
        # El siguiente evento es una salida de una solicitud del
        # sistema
        caso2(world)
    else:
        # El siguiente evento ocurre luego de la hora de cierre y 
        # ya no hay solicitudes en cola
        caso4(world)

Al ingresar los parámetros del sistema es posible simular uno o múltiples servidores. Posteriormente se obtienen las estadísticas de funcionamiento. 

In [ ]:
def simulador(T,ldp,lde,servidores):
    #se crea el sistema con todos los parametros
    world = SystemSim(T,ldp,lde,servidores)
    
    while(world.get_tiempoDisp()):
        #print("")
        #world.debug()
        selector(world)
    return world
    

In [ ]:
def get_data(world):
    #info del sistema
    print("INFORMACIÓN DEL SISTEMA: ")
    print("\t\tNa: ", world.Na,"\tNd: ", world.Nd)
    
    #solicitudes de cada servidor
    print("\nSOLICITUDES:")
    for i in range(len(world.servidores)):
        print("\t\tServidor ",i+1,"\t--> ",len(world.servidores[i].tasks))
        
    #tiempo ocupado de cada servidor
    tiempos_servidores = [] #lo almaceno para calcular IDDLE
    print("\nTIEMPO OCUPADOS:")
    for i in range(len(world.servidores)):
        #servidor X:
        tiempo_ocupado = 0
        for j in world.servidores[i].tasks:
            tiempo_ocupado += (j.t_salida - j.t_atendido)
        tiempos_servidores.append(tiempo_ocupado)
        print("\t\tServidor ",i+1,": ",tiempo_ocupado)
    
    #tiempo IDDLE
    print("\nTIEMPO DESOCUPADOS:")
    for i in range(len(tiempos_servidores)):
        print("\t\tServidor ",i+1,": ",(world.T - tiempos_servidores[i]))

    #Tiempo total de solicitudes en cola
    print("\nTIEMPO DE SOLICITUDES EN COLA:")
    tiempo_en_cola_total = 0
    solicitudes_en_cola = 0
    for i in range(len(world.servidores)):
        #servidor X:
        for j in world.servidores[i].tasks:
            tiempo_en_cola = (j.t_atendido - j.t_llegada)
            tiempo_en_cola_total += tiempo_en_cola
            if(tiempo_en_cola > 0):
                solicitudes_en_cola += 1
    print("\t\tTiempo total: ", tiempo_en_cola_total)
    print("\t\tTiempo promedio de cada solicitud: ", tiempo_en_cola_total/world.Na)
    print("\t\tPromedio de solicitudes por segundo: ", solicitudes_en_cola/world.T)
    
    
    #Promedio cuantas tasks en cola/seg
    #contadores = []
    #contador = 0
    #print("\nPROMEDIO DE SOLICITUDES EN COLA POR SEGUNDO:")
    ###Esta parte se tarda demasiado y no di como evaluarla de otra manera
    #se analizan los datos por intervalos de 1 segundo
    #for i in range(world.T+1):
        #contador = 0
        #for j in world.servidores:
            #for k in j.tasks:
                #if((k.t_llegada < i) and (k.t_llegada > i-1)):
                    #está en el intervalo evaluado, verifico que esperó en cola
                    #if((k.t_atendido - k.t_llegada)>0):
                        #estuvo en cola un tiempo en este intervalo
                        #contador +=1
                #elif((k.t_llegada < i) and (k.t_llegada < i-1)):
                    #estuvo esperando un tiempo en este intervalo
                    #if(k.t_atendido > i-1):
                        #contador +=1
                    #else (no es parte del intervalo)
        #contadores.append(contador)
    #print("\t\tPromedio de solicitudes en cola por segundo: ", contador/world.T)
    
    #tiempo de salida de la última solicitud
    print("\nTIEMPO DE SALIDA DE LA ÚLTIMA SOLICITUD: ")
    print("\t\tTiempo final: ",world.t)
    for i in range(len(world.servidores)):
        print("\t\tExtra Servidor ", i,": ",world.servidores[i].Tp)

<head><h1> SIMULACIÓN </h1></head>

<h3>Gorilla Megacomputing</h3> 
1 servidor<br>
lde = 100<br>
ldp = 2400/60

In [ ]:
simulacion = simulador(3600,2400/60,100,1)
print("---> Finished: SIMULACION 1 <---")
get_data(simulacion)

<h3>Ants smart computing</h3> 
10 servidores<br>
lde = 10<br>
ldp = 2400/60

In [ ]:
simulacion = simulador(3600,2400/60,10,10)
print("---> Finished: SIMULACION 2 <---")
get_data(simulacion)

<h3>Ants smart computing</h3> 
17 servidores<br>
lde = 10<br>
ldp = 2400/60

In [ ]:
simulacion = simulador(3600,2400/60,10,17)
print("---> Finished: SIMULACION 3 <---")
get_data(simulacion)

<h3>Gorilla Megacomputing</h3> 
1 servidor<br>
lde = 100<br>
ldp = 6000/60

In [ ]:
simulacion = simulador(3600,6000/60,100,1)
print("---> Finished: SIMULACION 4 <---")
get_data(simulacion)

<h3>Ants smart computing</h3> 
10 servidores<br>
lde = 10<br>
ldp = 6000/60

In [ ]:
simulacion = simulador(3600,6000/60,10,10)
print("---> Finished: SIMULACION 5 <---")
get_data(simulacion)